In [61]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch import nn
from sklearn.feature_selection import SelectKBest

In [62]:
TARGET_COLUMN = 'target'
METAINFO_COLUMNS = ['stage', 'move_count', 'weight']
METAINFO_COLUMNS_COUNT = len(METAINFO_COLUMNS)
STAGE_INDEX = 0
MOVE_COUNT_INDEX = 1
WEIGHT_INDEX = 2
TARGET_INDEX = -1

STARTPOS_STAGE = 24
INPUT_LAYER_SIZE = 64 * 12

BATCH_SIZE = 32768

In [63]:
dataset = np.zeros((0, INPUT_LAYER_SIZE + METAINFO_COLUMNS_COUNT + 1), np.int16)
for chunk in pd.read_csv('/home/wind-eagle/quirky_data/dataset/01.csv', chunksize=32768):
    dataset = np.append(dataset, chunk.values.astype(np.int16), axis=0)

In [68]:
class ClippedReLU(nn.Module):
    def __init__(self, clip_value: float):
        super(ClippedReLU, self).__init__()
        self.clip_value = clip_value

    def forward(self, x):
        return torch.clamp(x.relu(), max=self.clip_value)

In [69]:
class QNNE(nn.Module):
    def __init__(self):
        super(QNNE, self).__init__()

        self.opening = nn.Sequential()
        self.endgame = nn.Sequential()
        
        self.opening.add_module('linear1', nn.Linear(INPUT_LAYER_SIZE, 8))
        self.opening.add_module('clipped_relu1', ClippedReLU(1.0))
        self.opening.add_module('linear2', nn.Linear(8, 8))
        self.opening.add_module('clipped_relu2', ClippedReLU(1.0))
        self.opening.add_module('linear3', nn.Linear(8, 1))
        
        self.endgame.add_module('linear1', nn.Linear(INPUT_LAYER_SIZE, 8))
        self.endgame.add_module('clipped_relu1', ClippedReLU(1.0))
        self.endgame.add_module('linear2', nn.Linear(8, 8))
        self.endgame.add_module('clipped_relu2', ClippedReLU(1.0))
        self.endgame.add_module('linear3', nn.Linear(8, 1))

    def forward(self, x):
        opening_score = self.opening(x[:, METAINFO_COLUMNS_COUNT:TARGET_INDEX])
        endgame_score = self.endgame(x[:, METAINFO_COLUMNS_COUNT:TARGET_INDEX])
        score = (opening_score[:, 0] * (1.0 * x[:, STAGE_INDEX]) +\
                endgame_score[:, 0] * (24.0 - 1.0 * x[:, STAGE_INDEX])) / 24.0
        return score

In [72]:
def get_loss(model, X, y, w):
    y = (y + 1.0) / 2
    y_pred = 1.0 / (1.0 + torch.exp(-model(X)))
    assert y_pred.dim() == 1
    loss = torch.sum(w * ((y - y_pred) ** 2)) / torch.sum(w)
    return loss

In [73]:
model = QNNE()
opt = torch.optim.Adam(model.parameters(), lr=0.01)

history = []
train_loss = np.inf
full_train_loss = np.inf

progress = tqdm(np.arange(6000))
for i in progress:
    indices = np.random.randint(0, dataset.shape[0], BATCH_SIZE)
    X_batch = torch.tensor(dataset[indices], dtype=torch.float32)
    y_batch = torch.tensor(dataset[indices, TARGET_INDEX], dtype=torch.float32)
    w_batch = torch.tensor(dataset[indices, WEIGHT_INDEX], dtype=torch.float32)
    
    loss = get_loss(model, X_batch, y_batch, w_batch)

    loss.backward()
    
    opt.step()
    opt.zero_grad()

    history.append(loss.data.numpy())

    train_loss = np.mean(history[-40:])
    progress.set_description(f'Average batch loss = {train_loss:.6f}')

Average batch loss = 0.060627: 100%|██████████| 6000/6000 [12:24<00:00,  8.06it/s]


In [104]:
X = [np.array([8,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0\
               ])]
X = torch.tensor(X, dtype=torch.float32)
model(X)

tensor([-0.1371], grad_fn=<DivBackward0>)

In [101]:
def print_weights(f, arr):
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            print("{0:0.5f}".format(arr[i][j]), ' ', file=f, sep="", end="")

def print_biases(f, arr):
    for i in range(arr.shape[0]):
            print("{0:0.5f}".format(arr[i]), ' ', file=f, sep="", end="")

with open('model.qnne', 'w') as f:
    print_weights(f, model.opening.linear1.weight.T.detach().numpy())
    print_weights(f, model.opening.linear2.weight.T.detach().numpy())
    print_weights(f, model.opening.linear3.weight.T.detach().numpy())
    print_biases(f, model.opening.linear1.bias.detach().numpy())
    print_biases(f, model.opening.linear2.bias.detach().numpy())
    print_biases(f, model.opening.linear3.bias.detach().numpy())

    print_weights(f, model.endgame.linear1.weight.T.detach().numpy())
    print_weights(f, model.endgame.linear2.weight.T.detach().numpy())
    print_weights(f, model.endgame.linear3.weight.T.detach().numpy())
    print_biases(f, model.endgame.linear1.bias.detach().numpy())
    print_biases(f, model.endgame.linear2.bias.detach().numpy())
    print_biases(f, model.endgame.linear3.bias.detach().numpy())